<a href="https://colab.research.google.com/github/MrKyZs/Trabalho-4-semestre/blob/main/TDE_4_Semestre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
#import plotly.express as px
#pd.__version__

In [ ]:
path = "datasetcertinho.csv"
#Chama o método read_csv e a insere em um objeto da classe DataFrame nomeada r_df
#No metodo read_csv é usado como atributo o "path", que é o caminho para o arquivo, e o "sep" que é o modo de separação
#É usado o pd como módulo que chama o metodo read_csv
r_df = pd.read_csv(path, sep=';')

pathkek = "ipcakek.csv"
#Chama o método read_csv e a insere em um objeto da classe DataFrame nomeada kek_df
#No metodo read_csv é usado como atributo o "path", que é o caminho para o arquivo, e o "sep" que é o modo de separação
kek_df = pd.read_csv(pathkek, sep=';')

In [ ]:
#São escolhidas as colunas que serão passadas para o novo dataframe e as passa para o objeto columns
columns = r_df[['MES', 'ESTADO', 'PRODUTO', 'PRECO MEDIO REVENDA']]
#Passa essas colunas para o novo dataframe utilizando o metodo .copy()
df = columns.copy()
#Seleciona o objeto Series onde o nome da coluna selecionada é PRECO MEDIO REVENDA
#Transforma as virgulas dos números da coluna 'PRECO MEDIO REVENDA' em pontos usando str.replace e passando como atributos 
#O que será substituido pelo substituinte 
df['PRECO MEDIO REVENDA'] = df['PRECO MEDIO REVENDA'].str.replace(',','.')
#Seleciona os estados do dataset e os passa para a objeto numpy.ndarray nomeada como todos_estados e datas
todos_estados = np.unique(df['ESTADO'])
#datas = np.unique(df['MES'])

#Seleciona os estados que são usados para o gráfico e os coloca em uma list.
estados = ['SAO PAULO', 'RIO DE JANEIRO', 'ESPIRITO SANTO', 'MINAS GERAIS']
#estados = todos_estados
#estados = ['MARANHAO', 'PIAUI', 'CEARA', 'RIO GRANDE DO NORTE', 'PARAIBA', 'PERNAMBUCO', 'ALAGOAS', 'SERGIPE', 'BAHIA']

# Correlação de Pearson

In [ ]:
#Pega as operações mean e std utilizando uma função que recebe como atributos uma coluna e o dataframe. 
#É feito o calculo da media e do desvio usando os metodos .mean e .std, e os resultados são inserido em novas colunas 
#Que são criadas usando o nome da coluna passada como atributo da função mais o sufixo OP, onde é armazenado o valor da coluna original
#menos a media, dividido pelo desvio.
def operacao(col, df_completo):
  mean = df_completo[col].mean()
  std = df_completo[col].std()
  df_completo[col+' OP'] = (df_completo[col]-mean)/std
  return mean, std, df_completo

In [ ]:
#Gera a primeira coluna do dataframe final com o primeiro estado da array

#Pega apenas a as colunas que possuem PRODUTO = Gasolina Comum do primeiro estado selecionado, juntamente com o preço medio de revenda e passa para float usando
#O metodo .astype() que recebe como atributo o tipo que será convertido para. 
df_gas = df[(df['PRODUTO'] == 'GASOLINA COMUM')][df['ESTADO'] == estados[0]]
df_gas['PRECO MEDIO REVENDA'] = df_gas['PRECO MEDIO REVENDA'].astype(float)
#Utiliza o metodo .copy para copiar o kek_df para outro dataframe que sera usado aqui.
df_ip = kek_df.copy()
#Junta os meses e anos para fica no formato jan/20. Utiliza do .str.lower para passar a coluna para letras minusculas
#Adiciona a / que será usada na separação, e seleciona apenas os 2 ultimos digitos da coluna ano. Passando a coluna para string
#Com o metodo .astype(float) e a escolha dos dois ultimos caracteres é feito em .str[-2:]
df_ip['MES'] = kek_df["MES"].str.lower() + '/'+ kek_df['ANO'].astype(str).str[-2:]
#Utiliza do metodo .drop com atributos o nome da coluna e o axis.
#Esse drop retira a coluna indicada do dataframe.
df_ip = df_ip.drop('ANO', axis=1)
#Junta os datasets da gasolina e o do ipca utilizando o metodo pd.merge, que recebe como atributo ambos Dataframes
#Que serão juntados, e é passado também um atributo on='MES', que faz com que ambos DataFrames sejam juntos baseados
#Em suas colunas MES, ou seja, para toda linha em que os meses são iguais nos dataframes eles são juntos.
df_all = pd.merge(df_ip, df_gas, on='MES')
#Passa os valores na coluna UM MES para float utilizando str.replace para transformar as virgulas em pontos e o astype
#Que recebe como atributo o tipo que será o dado.
df_all['UM MES'] = df_all['UM MES'].str.replace(',','.').astype(float)
#Utiliza do metodo .drop com atributos o nome da coluna e o axis.
#Esse drop retira a coluna indicada do dataframe, nesse caso são as colunas que foram importadas do kek_df.
#Mas que não são usadas.
df_all = df_all.drop(['NUMERO INDICE','TRES MESES','SEIS MESES', 'UM ANO', 'DOZE MESES'], axis=1)

#Pega a variação percentual com o metodo pct_change() e faz * 100 para que os valores fiquem melhores para serem trabalhados.
variacao = df_all['PRECO MEDIO REVENDA'].pct_change()*100
#Cria uma coluna no dataframe para armazenar os valores das variações percentuais
df_all['PRECO MEDIO REVENDA VAR'] = variacao
#Armazena em uma objeto Series os valores da coluna UM MES do dataframe df_all
icpa = df_all['UM MES']

#Chama a função operação, passando como atributo a coluna PRECO MEDIO REVENDA e o dataframe df_all
#É feito o calculo da media e do desvio usando os metodos .mean e .std, e os resultados são inserido em novas colunas 
operacao('PRECO MEDIO REVENDA VAR', df_all)
#Chama a função operação, passando como atributo a coluna UM MES e o dataframe df_all
#É feito o calculo da media e do desvio usando os metodos .mean e .std, e os resultados são inserido em novas colunas 
operacao('UM MES', df_all)
#Cria a uma Series chamada pearson que vai armazenar a multiplicação entre a coluna PRECO MEDIO REVENDA OP
#Que é criada durante a chamada da função operacao, e da coluna UM MES OP, criada na chamada da funcao também
df_all['Pearson'] = df_all['PRECO MEDIO REVENDA VAR OP'] * df_all['UM MES OP']
#Cria uma Series chamada Pearson que será armazena os valores da coluna Pearson
pearson = df_all['Pearson']
#Utiliza do metodo .drop com atributos o nome da coluna e o axis.
#Esse drop retira a coluna indicada do dataframe, nesse caso são as colunas que não serão usadas no df_final.
#Mas que não são usadas.
df_final = df_all.drop(['PRECO MEDIO REVENDA VAR','UM MES OP', 'PRECO MEDIO REVENDA VAR OP', 'ESTADO', 'PRODUTO', 'PRECO MEDIO REVENDA', 'UM MES'], axis=1)
#Calcula a média dos indices de pearson gerados e coloca no df_final. Além de renomear as colunas utilizando o metodo
#Rename, que recebe como padrão o nome atual da coluna, seguido de dois pontos, e o que será renomeado para.
df_final = df_final.rename(columns={"Pearson": "ESTADO: %s | INDICE PEARSON %.4f" % (estados[0], pearson.mean())})
#Como no Series Pearson está o indice de correlação mensal, precisamos calcular a média de todas as linhas da Series
#É utilizado o metodo .mean() para isso.
Sudeste = pearson.mean()

#É realizado um for para iterar em todos os estados presentes na array "estados" a partir do primeiro item
#Que é feito em cima.
for estado in estados[1:]:
  second_df = df[(df['PRODUTO'] == 'GASOLINA COMUM')][df['ESTADO'] == estado]
  second_df['PRECO MEDIO REVENDA'] = second_df['PRECO MEDIO REVENDA'].astype(float)

  ipca_df2 = kek_df.copy()
  ipca_df2['MES'] = kek_df["MES"].str.lower() + '/'+ kek_df['ANO'].astype(str).str[-2:]
  df_usable = pd.merge(second_df,ipca_df2, on='MES')
  df_usable['UM MES'] = df_usable['UM MES'].str.replace(',','.').astype(float)
  df_usable = df_usable.drop(['NUMERO INDICE','TRES MESES','SEIS MESES', 'UM ANO', 'DOZE MESES', 'ANO'], axis=1)

  variacao = df_usable['PRECO MEDIO REVENDA'].pct_change()*100
  df_usable['PRECO MEDIO REVENDA VAR'] = variacao
  icpa = df_usable['UM MES']

  operacao('PRECO MEDIO REVENDA VAR', df_usable)
  operacao('UM MES', df_usable)
  df_usable['Pearson'] = df_usable['PRECO MEDIO REVENDA VAR OP'] * df_usable['UM MES OP']
  pearson = df_usable['Pearson']
  df_usable = df_usable.drop(['PRECO MEDIO REVENDA VAR','UM MES OP', 'PRECO MEDIO REVENDA VAR OP', 'ESTADO', 'PRODUTO', 'PRECO MEDIO REVENDA', 'UM MES'], axis=1)
  
  df_usable = df_usable.rename(columns={"Pearson": "ESTADO: %s | INDICE PEARSON %.4f" % (estado, pearson.mean())})
  df_final = df_final.merge(df_usable, on='MES')

  Sudeste = Sudeste + pearson.mean()

#Cria a ax que é um plot da biblioteca matplotlib, que recebe df_final jutamente com o metodo plot
#Que recebe o titulo do gráfico, o nome do eixo X, que também é a coluna que é usada para a plotagem
#E também o figsize, que é o tamanho do gráfico
#Abaixo também é indicado como IPCA o ylabel, usando o metodo set_ylabel.
ax = df_final.plot(title ="Correlação Gasolina com IPCA", x = 'MES', figsize=(20,10))
ax.set_ylabel("IPCA")
print(Sudeste/len(estados))